# <CENTER>БУСТИНГ

## <CENTER> Проект "Предсказание удовлетворенности полетом пассажиров"

Мы будем работать c данными, которые содержат результаты опроса клиентов авиакомпании по поводу их удовлетворённости полётом.
### Задача:
Предсказать удовлетворённость пассажиров.

In [16]:
import numpy as np
from numpy import random
import pandas as pd
import sys
import re
import statistics
import json
from itertools import combinations
from pylab import rcParams

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
from scipy.stats import ttest_ind
from scipy.stats import randint
from scipy.stats import bernoulli
from scipy.stats import norm, uniform

from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn import cluster
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from catboost import Pool, CatBoostClassifier
from catboost.utils import get_confusion_matrix
!pip install xgboost
!pip install catboost

In [2]:
df = pd.read_csv("data/AirPass.csv", index_col=0)
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


Познакомимся с данными:

- Gender — пол пассажира (женский, мужской);
- Customer Type — тип клиента (постоянный/непостоянный клиент);
- Age — возраст клиента;
- Type of Travel — цель перелета (личная/деловая поездка);
- Class — туристический класс пассажира (Business, Eco, Eco Plus);
- Flight distance — расстояние полета;
- Inflight wifi service — уровень удовлетворённости Wi-Fi (0 — не применимо, 1–5);
- Departure/Arrival time convenient — уровень удовлетворённости временем отправления и прибытия;
- Ease of Online booking — уровень удовлетворённости онлайн-бронированием;
- Gate location — уровень удовлетворённости расположением выхода на посадку;
- Food and drink — уровень удовлетворённости едой и напитками;
- Online boarding — уровень удовлетворённости онлайн-регистрацией;
- Seat comfort — уровень удовлетворённости комфортом сидений;
- Inflight entertainment — уровень удовлетворённости развлечениями на борту;
- On-board service — уровень удовлетворённости сервисом на борту;
- Leg room service — уровень удовлетворённости местом для ног;
- Baggage handling — уровень удовлетворённости обработкой багажа;
- Check-in service — уровень удовлетворённости услугами регистрации;
- Inflight service — уровень удовлетворённости обслуживанием во время полёта;
- Cleanliness — уровень удовлетворённости чистотой;
- Departure Delay in Minutes — задержка при отправлении (в минутах);
- Arrival Delay in Minutes — задержка при прибытии (в минутах);
- Satisfaction — удовлетворённость авиакомпанией — целевая переменная (satisfaction/neutral/dissatisfaction).

Для начала сделаем небольшую предобработку данных. И найдем пропущенные значения

In [4]:
df.isnull().sum()

id                                     0
Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             310
satisfaction                           0
dtype: int64

**Теперь давайте избавимся от найденных пропусков. Заполним их все медианными значениями. После этого вычислим среднее арифметическое для признака, отражающего задержку при прибытии в минутах.**

In [5]:
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median())
df['Arrival Delay in Minutes'].mean()

15.133392362180475

**Проведём небольшой разведывательный анализ. Посмотрим, в каких категориях пассажиров превалировали удовлетворённые полетом клиенты.**

Сравним удовлетворённость полётом мужчин и женщин.

In [7]:
df.groupby('Gender')['satisfaction'].value_counts()*100/df.shape[0]

Gender  satisfaction           
Female  neutral or dissatisfied    29.058554
        satisfied                  21.687327
Male    neutral or dissatisfied    27.608177
        satisfied                  21.645942
Name: satisfaction, dtype: float64

Теперь сравним категории пассажиров в зависимости от их цели поездки.

In [8]:
df.groupby('Type of Travel')['satisfaction'].value_counts()*100/df.shape[0]

Type of Travel   satisfaction           
Business travel  satisfied                  40.177472
                 neutral or dissatisfied    28.785225
Personal Travel  neutral or dissatisfied    27.881506
                 satisfied                   3.155798
Name: satisfaction, dtype: float64

Давайте посмотрим в каком туристическом классе наибольший процент довольных клиентов?

In [9]:
df.groupby('Class')['satisfaction'].value_counts()*100/df.shape[0]

Class     satisfaction           
Business  satisfied                  33.184478
          neutral or dissatisfied    14.614452
Eco       neutral or dissatisfied    36.614567
          satisfied                   8.374076
Eco Plus  neutral or dissatisfied     5.437712
          satisfied                   1.774715
Name: satisfaction, dtype: float64

**Перекодируем часть бинарных признаков, чтобы использовать их при обучении:**

In [10]:
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

**Для остальных категориальных признаков создадим dummy-переменные. Сделаем это с помощью функции get_dummies() из библиотеки Pandas.**

In [11]:
df=pd.get_dummies(df)
df.shape

(103904, 26)

**Разбъем данные на обучающую и тестовую выборки в соотношении 80/20, параметр random_state = 26.**

In [12]:
X = df.drop(['satisfaction'], axis=1)
y = df['satisfaction']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=26)
y_test.shape

(20781,)

## Масштабирование

**Обучим на обучающей выборке метод StandardScaler() и с помощью него преобразуем и обучающую, и тестовую выборки. 
Примечание. Отметим, что если бы дальше мы работали только с деревьями, масштабирование бы не требовалось. Однако мы реализуем его, чтобы можно было обучать и другие модели и сравнивать полученные результаты.
В качестве ответа введем самое первое значение из матрицы преобразованных признаков тестовой выборки.**

In [14]:
scaler = StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 
X_test[0][0]

0.9408251379303

## Логистическая регрессия

**Перейдём к обучению моделей. В качестве первой модели возьмём самую простую — логистическую регрессию. Мы делаем это для того, чтобы потом сравнивать с ней полученные результаты: так мы сможем выяснить, насколько ансамбли смогут улучшить точность прогноза.
Обучим логистическую регрессию с параметрами по умолчанию на наших данных. В качестве ответа введем значение метрики f1_score.**

In [17]:
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)
preds_test = model_lr.predict(X_test)
f1_score(preds_test, y_test)

0.8546883773161146

## БУСТИНГ

### AdaBoost

Начнём с обучения первой модели — **AdaBoost**. В качестве базовой модели для неё возьмем решающее дерево с параметром random_state = 26. Обучите AdaBoost, зафиксировав random_state со значением 26 и задав темп обучения 0.01. В качестве ответа введем значение метрики f1_score.

In [18]:
model_ada = AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=26),random_state=26,learning_rate=0.01)

model_ada.fit(X_train, y_train)
preds_test = model_ada.predict(X_test)
f1_score(preds_test, y_test)

0.9404794558121674

### GradientBoosting

Перейдем к следующему алгоритму — **градиентному бустингу**. Будем настраивать количество деревьев и темп обучения, делая перебор по следующей сетке: Используем для поиска оптимальных параметров `GridSearchCV`, а для ускорения работы алгоритма зададим параметр кросс-валидации, равный `3`. И посмотрим наибольшее значение метрики `f1_score`.

In [20]:
model_for_gs = GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
gs = GridSearchCV(model_for_gs, 
                  params, 
                  cv=3, 
                  scoring=make_scorer(f1_score),
                  verbose=5)
 
gs.fit(X_train, y_train)
 
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.873 total time=   0.1s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.870 total time=   0.1s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.871 total time=   0.1s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.880 total time=   0.2s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.878 total time=   0.2s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.875 total time=   0.2s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.901 total time=   0.3s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.896 total time=   0.4s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.897 total time=   0.4s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   0.7s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   0.7s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;

### XGBoost

Обучим алгоритм **XGBoost**. Так как он достаточно мощный «из коробки», определим его с параметрами по умолчанию. Зададим только random_state = 26. Полсмотрим какое значение метрики f1_score получилось. Для того чтобы обучить алгоритм XGBoost для решения задачи классификации, нам понадобится `XGBClassifier` из библиотеки `xgboost`, установленной ранее. Вся дальнейшая последовательность действий (обучение модели, предсказание, оценка качества) идентична другим алгоритмам, например логистической регрессии.

In [21]:
model_xgb = XGBClassifier(random_state=26)
model_xgb.fit(X_train,y_train)
preds_test = model_xgb.predict(X_test)
f1_score(preds_test, y_test)

0.9579785161685312

### CatBoost

Обучим алгоритм **CatBoost**. Как и `XGBoost`, будем обучать его с настройками по умолчанию. Посмотрим какое значение метрики f1_score получилось. Для того чтобы обучить алгоритм `CatBoost`, нам понадобится `CatBoostClassifier()` из библиотеки `catboost`, установленной ранее. Вся дальнейшая последовательность действий (обучение модели, предсказание, оценка качества) идентична другим алгоритмам, например логистической регрессии.

In [22]:
model = CatBoostClassifier(random_state=26)
model.fit(X_train, y_train)
preds_class = model.predict(X_test)
f1_score(preds_class, y_test)

Learning rate set to 0.068023
0:	learn: 0.6018089	total: 71.6ms	remaining: 1m 11s
1:	learn: 0.5020769	total: 78.5ms	remaining: 39.1s
2:	learn: 0.4472481	total: 84.6ms	remaining: 28.1s
3:	learn: 0.4028675	total: 92.6ms	remaining: 23s
4:	learn: 0.3674724	total: 101ms	remaining: 20.1s
5:	learn: 0.3397844	total: 110ms	remaining: 18.3s
6:	learn: 0.3121211	total: 118ms	remaining: 16.8s
7:	learn: 0.2917499	total: 127ms	remaining: 15.8s
8:	learn: 0.2749039	total: 136ms	remaining: 14.9s
9:	learn: 0.2575191	total: 144ms	remaining: 14.2s
10:	learn: 0.2473690	total: 152ms	remaining: 13.6s
11:	learn: 0.2377531	total: 159ms	remaining: 13.1s
12:	learn: 0.2279309	total: 168ms	remaining: 12.7s
13:	learn: 0.2212512	total: 176ms	remaining: 12.4s
14:	learn: 0.2100359	total: 184ms	remaining: 12.1s
15:	learn: 0.2025733	total: 192ms	remaining: 11.8s
16:	learn: 0.1942303	total: 201ms	remaining: 11.6s
17:	learn: 0.1877939	total: 210ms	remaining: 11.5s
18:	learn: 0.1832381	total: 218ms	remaining: 11.3s
19:	lear

0.960182404626849

**Выведем матрицу ошибок для алгоритма, который получил наилучшие показатели качества модели на обучающей выборке (будем считать, что оцениваем по f1_score). Заполним соответствующие значения:
Для того чтобы построить матрицу ошибок в CatBoost, необходимо использовать следующий шаблон:
get_confusion_matrix(модель, Pool(признаки обучающей выборки, целевая переменная обучающей выборки))**

In [23]:
cm = get_confusion_matrix(model, Pool(X_train, y_train))
cm

array([[46661.,   545.],
       [ 1274., 34643.]])

**Оценим важность признаков для модели из предыдущего задания. Отметим три признака, которые оказывают наибольшее влияние на значение целевой переменной:**

In [24]:
clf = CatBoostClassifier(verbose=False, random_state=26)
clf.fit(X_train, y_train)

In [25]:
y_pred = clf.predict(X_test)
round(f1_score(y_pred, y_test), 3)

0.96

In [26]:
pd.DataFrame(
    {
        "feature_importance": clf.get_feature_importance(),
        "feature_names": df.drop(columns="satisfaction").columns,
    }
).sort_values(by=["feature_importance"], ascending=False)

,feature_importance,feature_names
6,25.364737,Inflight wifi service
4,18.391876,Type of Travel
11,7.401483,Online boarding
2,7.240564,Customer Type
22,5.420057,Class_Business
17,3.925791,Checkin service
3,3.742350,Age
16,3.640798,Baggage handling
9,3.196286,Gate location
12,3.012455,Seat comfort
